In [1]:
from libs.alibaba.alibaba import Alibaba
from libs.alibaba.inquiry import Inquiry
from libs.alibaba.inquiry import element_has_css_class
from libs.email import Email
from libs.json import JSON

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

from string import Template
from datetime import datetime
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq

import json
import requests
import time
import re
import math

import types
import traceback
import threading

import pendulum

import logging
from logging.handlers import TimedRotatingFileHandler

In [12]:
market = JSON.deserialize('.', 'storage', 'markets.json')['Tools']
market

. storage markets.json
.\storage\markets.json was deserialized!


{'name': 'Tools',
 'directory': 'E:\\glitter\\Tools',
 'lid': 'glittertools@glittergroupcn.com',
 'lpwd': 'Songjianwei68970747',
 'accounts': [],
 'homepage': 'https://glittertools.en.alibaba.com',
 'lname': 'Jessica Chen'}

In [3]:
inquiry = Inquiry(market)
# inquiry.login()

E:\glitter\Eyelashes_config [] inquiry_tracking_ids_Jeremy.json
E:\glitter\Eyelashes_config\inquiry_tracking_ids_Jeremy.json was deserialized!


In [ ]:
def webww_check(self):
    # try to login
    icon = WebDriverWait(self.browser, 15).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#webatm2-iconbar')))
    container = self.browser.find_element_by_css_selector('#webww-contacts')
    dialog_iframe = self.browser.find_element_by_css_selector('#webatm2-iframe')
    while True:
        icon.click()
        WebDriverWait(self.browser, 15).until(EC.invisibility_of_element_located((By.CSS_SELECTOR, '#webww-contacts .webatm2-tips')))
        confirm = self.browser.find_element_by_css_selector('#webww-contacts .webatm2-confirm')
        if confirm.is_displayed():
            # failed login, cancel checking
            print('Trade Manager has been loged in somewhere else, cancel checking')
            close_btn = confirm.find_element_by_css_selector('span.webatm2-confirm-close')
            close_btn.click()
            return
        
        if 'show-panel' in container.get_attribute('class'):
            break
    
    panel = WebDriverWait(self.browser, 15).until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, '#webatm2-panel')))
    # after login
    
    # open dialog
    tab_sysmsg = panel.find_element_by_css_selector('div.tab-sysmsg')
    tab_sysmsg.click()
    panel.find_element_by_css_selector('.panel-sysmsg.active a').click()
    WebDriverWait(self.browser, 15).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#webatm2-dialog')))
    
    # switch to iframe and navigate to recent threads
    self.browser.switch_to.frame(dialog_iframe)
    sidebar = self.browser.find_element_by_css_selector('.webatm2-sidebar')
    head = self.browser.find_element_by_css_selector('.webatm2-window-head')
    body = self.browser.find_element_by_css_selector('.webatm2-window-body')
    sidebar.find_element_by_css_selector('.tab-thread').click()
    time.sleep(1)
    doc=pq(sidebar.find_element_by_css_selector('.webatm2-thread-list').get_attribute('innerHTML'))
    for div in doc.find('.webatm2-thread'):
        div = pq(div)
        thread = {}
        thread['id'] = div.attr('data-user-id')
        thread['online'] = div.hasClass('online')
        thread['name'] = div.find('.thread-info .name').text()
        thread['unread'] = div.find('.thread-info span.unread').text()
        print(thread)
        
        if thread['id'] == 'enaliintcl1113504018jszx' or thread['id'] == 'enaliintpe1318961315wklp':
            self.webww_switch_to_thread(thread, sidebar, head, body)
            
    self.browser.switch_to.default_content()
inquiry.webww_check = types.MethodType(webww_check, inquiry)

In [ ]:
def webww_switch_to_thread(self, thread, sidebar, head, body):
    
    div = sidebar.find_element_by_css_selector('div.webatm2-thread[data-user-id="'+thread['id']+'"]')
    
    if 'active' not in div.get_attribute('class'):
        msgs = body.find_elements_by_css_selector('.webatm2-messages .webatm2-message')
        div.click()
    
        if msgs:
            WebDriverWait(self.browser, 15).until(EC.staleness_of(msgs[-1]))
        else:
            WebDriverWait(self.browser, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.webatm2-messages .webatm2-message')))
        time.sleep(1)
    WebDriverWait(self.browser, 15).until(
            EC.text_to_be_present_in_element((By.CSS_SELECTOR, '.webatm2-window-head .user-name'), thread['name']))

    doc = pq(self.browser.find_element_by_css_selector('.webatm2-window-body .webatm2-messages').get_attribute('innerHTML'))
    messages = []
    for div in doc.find('.webatm2-message'):
        div = pq(div)
        message = {}
        message['is_self'] = div.hasClass('self')
        message['datetime'] = div.find('.time-tag').text() if div.find('.time-tag') else None
        message['content'] = div.find('.content').html()
        messages.append(message)
        
#     self.browser.find_element_by_css_selector('#webatm2-dialog .webatm2-dialog-close').click()
    print()
    print(messages)
inquiry.webww_switch_to_thread = types.MethodType(webww_switch_to_thread, inquiry)

In [ ]:
def webww_send_message(self, message):
    pass

inquiry.webww_send_message = types.MethodType(webww_send_message, inquiry)

In [ ]:
inquiry.browser.switch_to.default_content()

In [ ]:
inquiry.load_url()

In [ ]:
inquiry.load_url()
inquiry.webww_check()

In [6]:
def send_message(self, message, attach=None):
    chat_form = self.browser.find_element_by_css_selector('form.reply-wrapper')
    chat_form.click()

    uploading_failed = False
    if attach:
        file_input = self.browser.find_element_by_css_selector('input#attachs')
        file_input.send_keys(attach)

        css = 'div.next-upload-list-item:last-child'
        div = WebDriverWait(self.browser, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, css)))
        try:
            WebDriverWait(self.browser, 120).until(
                element_has_css_class(div, 'next-upload-list-item-done'))
        except TimeoutException:
            print('time out, cancel uploading')
            div.find_element_by_css_selector('i.next-icon-close').click()
            uploading_failed = True
        
    if uploading_failed:
        return False
            
#     if uploading_failed or attach is None:
#         doc = pq(message)
#         doc.find('span.attach').remove()
#         message = doc.outer_html()

    WebDriverWait(self.browser, 15).until(
        EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, '#inquiry-content_ifr')))
    body = self.browser.find_element_by_tag_name('body')
    body.click()

    js = self.reply_js_template.format(message=message)
    self.browser.execute_script(js)
    self.browser.switch_to.default_content()

    time.sleep(1)

    btn_send = chat_form.find_element_by_css_selector('button.send')
    btn_send.click()
    return True

inquiry.send_message = types.MethodType(send_message, inquiry)

In [7]:
def reply(self, enquiry):
    try:
        self.open_inquiry_in_new_tab(enquiry)
        conversation = self.get_conversation()

        messages = conversation['messages']
        buyer = conversation['buyer']

        last_buyer_messages = []
        is_replied = False
        reply_count = 0
        for msg in messages:
            if msg['position'] == 'left':
                last_buyer_messages.append(msg)
                is_replied = False
            elif msg['position'] == 'right' and 'Seller Assistant' not in msg['user']:
                last_buyer_messages = []
                is_replied = True
                reply_count += 1

        eid = enquiry['id']
        tracking = self.tracking_ids[eid]
        last_status = tracking['status'][-1]
        if last_status == 'new':
            greetings = '<p>Pleased to hear from you.</p><br>'
            # buyer['email'] = 'changshu.qd@gmail.com'  # for testing purpose
            buyer['email'] = ''  # for testing purpose
            if buyer['email']:
                emails = [buyer['email']]
                mime_message = Email.message_of_product_catalog(self.market, self.account, buyer['name'])
                if Email.send(self.account, emails, mime_message):
                    params = {'buyer': buyer['name'], 'greetings': greetings, 'email': buyer['email'],
                              'sender': self.lname}
                    message = self.reply_templates['notify_catalog_was_sent'].substitute(params)
                    if self.send_message(message, self.catalog):
                        tracking['status'].append('catalog was sent by email')
                        self.save_tracking_ids()
            else:
                params = {'buyer': buyer['name'], 'greetings': greetings, 'sender': self.lname}
                message = self.reply_templates['notify_first_reply_without_email'].substitute(params)
                if self.send_message(message, self.catalog):
                    tracking['status'].append('catalog was sent directly')
                    self.save_tracking_ids()
        elif 'catalog was sent' in last_status:
            pass
        elif 'wait for email address' in last_status:
            text = ''
            for msg in last_buyer_messages:
                text = text + '\n\n ' + msg['content']
            emails = re.findall('([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)', text)
            print(emails)
            if emails:
                tracking['emails'] = emails
                self.save_tracking_ids()
                mime_message = Email.message_of_product_catalog(self.market, self.account, buyer['name'])
                if Email.send(self.account, emails, mime_message):
                    greetings = ''
                    params = {'buyer': buyer['name'], 'greetings': greetings, 'email': ', '.join(emails),
                              'sender': self.lname}
                    message = self.reply_templates['notify_catalog_was_sent'].substitute(params)
                    if self.send_message(message, self.catalog):
                        tracking['status'].append('catalog was sent by email')
                        self.save_tracking_ids()
    except Exception as e:
        self.notify("danger", '回复 询盘 [' + enquiry['id'] + '] 时 发生错误! ' + str(e))
        traceback.print_exc()
    finally:
        self.close_inquiry_and_switch_back()
inquiry.reply = types.MethodType(reply, inquiry)

In [8]:
def is_auto_reply_needed(self, enquiry):
    eid = enquiry['id']
    # #for testing purpose
    if enquiry['id'] == '11964562469':
        self.tracking_ids[eid] = {}
        self.tracking_ids[eid]['datetime'] = pendulum.now()
        self.tracking_ids[eid]['status'] = ['new']
        self.save_tracking_ids()
        return True
    else:
        return False

    if enquiry['responsible_person'].lower() != self.lname.lower():
        return False

    if eid in self.tracking_ids and self.tracking_ids[eid]['datetime'].diff().in_hours() <= 12:
        if not enquiry['is_replied']:
            return True
    if len(enquiry['tags']) == 0 and enquiry['is_new']:
        if eid not in self.tracking_ids:
            self.tracking_ids[eid] = {}
            self.tracking_ids[eid]['datetime'] = pendulum.now()
            self.tracking_ids[eid]['status'] = ['new']
            self.save_tracking_ids()
        return True
    return False
inquiry.is_auto_reply_needed = types.MethodType(is_auto_reply_needed, inquiry)

In [4]:
inquiry.load_url()
inquiries = inquiry.get_inquiries()

primary 打开网址：https://i.alibaba.com
primary 等待登陆页面加载 ... ...
primary 输入登录信息， 并登录
success 成功 登录 阿里巴巴 国际站！用户名：Jeremy


In [5]:
enquiry = inquiries[1]
enquiry

{'url': '/message/default.htm#feedback/detail/all/11964562469/11964562469/MC1IDX1pHHtVT6vmZzq1IpMGODGI9ftGKNPj7X9IQ3lDd4DIQIlNV5P9IQ-TbWi_eVhUC53/',
 'id': '11964562469',
 'date': '2018-7-18',
 'tags': [],
 'is_replied': True,
 'title': 'I want to buy the product you are selling on Alibaba.com.',
 'buyer': 'prestl bernhard',
 'is_buyer_online': False,
 'buyer_country': 'United States',
 'buyer_local_time': '19: 33 Wed Jul 18',
 'responsible_person': 'Jeremy Wang',
 'status': '洽谈中',
 'is_new': False}

In [9]:
inquiry.is_auto_reply_needed(enquiry)

E:\glitter\Eyelashes_config\inquiry_tracking_ids_Jeremy.json was serialized!


True

In [10]:
inquiry.reply(enquiry)

E:\glitter\Eyelashes_config\inquiry_tracking_ids_Jeremy.json was serialized!


In [ ]:
inquiry.load_reply_templates()
inquiry.reply_templates['notify_first_reply_without_email']

In [ ]:
inquiry.tracking_ids['11947313067']['status'] = ['new']
inquiry.tracking_ids

In [ ]:
inquiry.check()

In [11]:
inquiry.tracking_ids

{'11964562469': {'datetime': DateTime(2018, 7, 19, 10, 34, 0, 614427, tzinfo=Timezone('Asia/Shanghai')),
  'status': ['new', 'catalog was sent directly']}}

In [ ]:
inquiry.close_inquiry_and_switch_back()

In [ ]:
inquiry.load_find_catalog()

In [ ]:
inquiry.catalog

In [ ]:
enquiry = inquiries[8]
enquiry

In [ ]:
enquiry = inquiries[8]
inquiry.open_inquiry_in_new_tab(enquiry)
conversation = inquiry.get_conversation()
inquiry.close_inquiry_and_switch_back()
messages = conversation['messages']
buyer = conversation['buyer']
messages

In [ ]:
enquiry = inquiries[8]
enquiry

In [ ]:
# inquiry.load_tracking_ids()
inquiry.tracking_ids

In [ ]:
inquiry.close_inquiry_and_switch_back()

In [ ]:
# inquiry.close_inquiry_and_switch_back()
# inquiry.tracking_ids['11947313067']['status'] = ['new']
enquiry = inquiries[0]
inquiry.reply(enquiry)

In [ ]:
inquiry.tracking_ids

In [ ]:
def try_login_webww():
    btn_login = WebDriverWait(inquiry.browser, 15).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#webatm2-iconbar')))
    btn_login.click()

In [ ]:
panel = inquiry.browser.find_element_by_css_selector('#webatm2-panel')

In [ ]:
inquiries[11]

In [ ]:
re.findall('([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)', text)

In [ ]:
inquiry.browser.save_screenshot("screenshot.png")